In [58]:
from torchnlp.datasets import snli_dataset
import numpy as np
import nltk
import torch


# premise::hypothesis::label::premise_transitions::hypothesis_transitions
train_loader = snli_dataset(train=True)
val_loader = snli_dataset(dev=True)
test_loader = snli_dataset(test=True)

In [59]:
glove = {}
f = open('./data/glove.6B.300d.txt', 'r')
for line in f:
    splitLine = line.split()
    word = splitLine[0]
    embedding = np.array([float(val) for val in splitLine[1:]])
    glove[word] = embedding
    
word_dict = {'p*a*d':0, 'u*n*k':1}
word_list = ['p*a*d', 'u*n*k']
for word in glove.keys():
    word_dict[word] = len(word_list)
    word_list.append(word)
    
np.save('./data/glove_dict.npy', np.array(list(glove.values())))
np.save('./data/word_dict.npy', word_dict)
np.save('./data/word_list.npy', word_list)

In [60]:
train_x = []
train_y = []
train_r = []
for data in train_loader:
    p = nltk.word_tokenize(data['premise'].lower())
    h = nltk.word_tokenize(data['hypothesis'].lower())
    l = 1 if data['label'] == 'neutral' or data['label'] == 'neutral' else\
        0 if data['label'] == 'contradiction' else None
    
    if l is None:
        continue
    
    train_x.append([word_dict[w] if word_dict.get(w)!=None else 1 for w in p])
    train_y.append([word_dict[w] if word_dict.get(w)!=None else 1 for w in h])
    train_r.append(l)

In [61]:
val_x = []
val_y = []
val_r = []
for data in val_loader:
    p = nltk.word_tokenize(data['premise'].lower())
    h = nltk.word_tokenize(data['hypothesis'].lower())
    l = 1 if data['label'] == 'neutral' or data['label'] == 'neutral' else\
        0 if data['label'] == 'contradiction' else None
    
    if l is None:
        continue
    
    val_x.append([word_dict[w] if word_dict.get(w)!=None else 1 for w in p])
    val_y.append([word_dict[w] if word_dict.get(w)!=None else 1 for w in h])
    val_r.append(l)

In [62]:
test_x = []
test_y = []
test_r = []
for data in test_loader:
    p = nltk.word_tokenize(data['premise'].lower())
    h = nltk.word_tokenize(data['hypothesis'].lower())
    l = 1 if data['label'] == 'neutral' or data['label'] == 'neutral' else\
        0 if data['label'] == 'contradiction' else None
    
    if l is None:
        continue
    
    test_x.append([word_dict[w] if word_dict.get(w)!=None else 1 for w in p])
    test_y.append([word_dict[w] if word_dict.get(w)!=None else 1 for w in h])
    test_r.append(l)

In [63]:
max_x = 0
for x in train_x+val_x+test_x:
    if max_x < len(x):
        max_x = len(x)
        
max_y = 0
for y in train_y+val_y+test_y:
    if max_y < len(y):
        max_y = len(y)
        
print(max_x, max_y)

82 62


In [64]:
train_data = []
for x, y, l in zip(train_x, train_y, train_r):
    len_x = len(x)
    len_y = len(y)
    x = x + [0] * (max_x-len_x) if len_x < max_x else x
    y = y + [0] * (max_y-len_y) if len_y < max_y else y
    train_data.append([x,len_x,y,len_y,l])
    
val_data = []
for x, y, l in zip(val_x, val_y, val_r):
    len_x = len(x)
    len_y = len(y)
    x = x + [0] * (max_x-len_x) if len_x < max_x else x
    y = y + [0] * (max_y-len_y) if len_y < max_y else y
    val_data.append([x,len_x,y,len_y,l])
    
test_data = []
for x, y, l in zip(test_x, test_y, test_r):
    len_x = len(x)
    len_y = len(y)
    x = x + [0] * (max_x-len_x) if len_x < max_x else x
    y = y + [0] * (max_y-len_y) if len_y < max_y else y
    test_data.append([x,len_x,y,len_y,l])

In [65]:
np.save('./data/train.npy', train_data)
np.save('./data/val.npy', val_data)
np.save('./data/test.npy', test_data)

In [66]:
a=b=0
for r in train_r:
    if r==0:
        a+=1
    elif r==1:
        b+=1
print(a,b)

183187 182764
